# 文本特征及分类

## 1. 情感分析

In [1]:
# 简单的例子

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier

text1 = 'I like the movie so much!'
text2 = 'That is a good movie.'
text3 = 'This is a great one.'
text4 = 'That is a really bad movie.'
text5 = 'This is a terrible movie.'

def proc_text(text):
    """
        预处处理文本
    """
    # 分词
    raw_words = nltk.word_tokenize(text)
    
    # 词形归一化
    wordnet_lematizer = WordNetLemmatizer()    
    words = [wordnet_lematizer.lemmatize(raw_word) for raw_word in raw_words]
    
    # 去除停用词
    filtered_words = [word for word in words if word not in stopwords.words('english')]
    
    # True 表示该词在文本中，为了使用nltk中的分类器
    return {word: True for word in filtered_words}

# 构造训练样本
train_data = [[proc_text(text1), 1],
              [proc_text(text2), 1],
              [proc_text(text3), 1],
              [proc_text(text4), 0],
              [proc_text(text5), 0]]

print(train_data)

[[{'I': True, 'like': True, 'movie': True, 'much': True, '!': True}, 1], [{'That': True, 'good': True, 'movie': True, '.': True}, 1], [{'This': True, 'great': True, 'one': True, '.': True}, 1], [{'That': True, 'really': True, 'bad': True, 'movie': True, '.': True}, 0], [{'This': True, 'terrible': True, 'movie': True, '.': True}, 0]]


In [2]:
# 训练模型
nb_model = NaiveBayesClassifier.train(train_data)

# 测试模型
text6 = 'That is a bad one.'
print(nb_model.classify(proc_text(text6)))

0


## 2. 文本相似度

In [3]:
import nltk
from nltk import FreqDist

text1 = 'I like the movie so much '
text2 = 'That is a good movie '
text3 = 'This is a great one '
text4 = 'That is a really bad movie '
text5 = 'This is a terrible movie'

text = text1 + text2 + text3 + text4 + text5
words = nltk.word_tokenize(text)
freq_dist = FreqDist(words)
print(freq_dist['That'])

2


In [4]:
# 取出常用的n=5个单词
n = 5

# 构造“常用单词列表”
most_common_words = freq_dist.most_common(n)
print(most_common_words)

[('movie', 4), ('is', 4), ('a', 4), ('That', 2), ('This', 2)]


In [5]:
def lookup_pos(most_common_words):
    """
        查找常用单词的位置
    """
    result = {}
    pos = 0
    for word in most_common_words:
        result[word[0]] = pos
        pos += 1
    return result

# 记录位置
std_pos_dict = lookup_pos(most_common_words)
print(std_pos_dict)

{'movie': 0, 'is': 1, 'a': 2, 'That': 3, 'This': 4}


In [6]:
# 新文本
new_text = 'That one is a good movie. This is so good!'

# 初始化向量
freq_vec = [0] * n

# 分词
new_words = nltk.word_tokenize(new_text)

# 在“常用单词列表”上计算词频
for new_word in new_words:
    if new_word in list(std_pos_dict.keys()):
        freq_vec[std_pos_dict[new_word]] += 1

print(freq_vec)

[1, 2, 1, 1, 1]


## 3. 文本分类及TF-IDF

### 3.1 NLTK中的TF-IDF

In [7]:
from nltk.text import TextCollection

text1 = 'I like the movie so much '
text2 = 'That is a good movie '
text3 = 'This is a great one '
text4 = 'That is a really bad movie '
text5 = 'This is a terrible movie'

# 构建TextCollection对象
tc = TextCollection([text1, text2, text3, 
                        text4, text5])
new_text = 'That one is a good movie. This is so good!'
word = 'That'
tf_idf_val = tc.tf_idf(word, new_text)
print('{}的TF-IDF值为：{}'.format(word, tf_idf_val))

That的TF-IDF值为：0.02181644599700369


### 3.2 sklearn中的TF-IDF

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
feat = vectorizer.fit_transform([text1, text2, text3, text4, text5])

In [41]:
vectorizer.get_feature_names()

['bad',
 'good',
 'great',
 'is',
 'like',
 'movie',
 'much',
 'one',
 'really',
 'so',
 'terrible',
 'that',
 'the',
 'this']

In [42]:
feat_array = feat.toarray()
feat_array.shape

(5, 14)

In [43]:
feat_array[0, :]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.48127008,
        0.27113917,  0.48127008,  0.        ,  0.        ,  0.48127008,
        0.        ,  0.        ,  0.48127008,  0.        ])

In [46]:
vectorizer.transform([new_text]).toarray()

array([[ 0.        ,  0.67082255,  0.        ,  0.37792972,  0.        ,
         0.18896486,  0.        ,  0.33541128,  0.        ,  0.33541128,
         0.        ,  0.27060771,  0.        ,  0.27060771]])

### 3.3 中文TF-IDF

In [51]:
ch_text1 = ' 非常失望，剧本完全敷衍了事，主线剧情没突破大家可以理解，可所有的人物都缺乏动机，正邪之间、妇联内部都没什么火花。团结-分裂-团结的三段式虽然老套但其实也可以利用积攒下来的形象魅力搞出意思，但剧本写得非常肤浅、平面。场面上调度混乱呆板，满屏的铁甲审美疲劳。只有笑点算得上差强人意。'
ch_text2 = ' 2015年度最失望作品。以为面面俱到，实则画蛇添足；以为主题深刻，实则老调重弹；以为推陈出新，实则俗不可耐；以为场面很high，实则high劲不足。气！上一集的趣味全无，这集的笑点明显刻意到心虚。全片没有任何片段给我有紧张激动的时候，太弱了，跟奥创一样。'
ch_text3 = ' 《铁人2》中勾引钢铁侠，《妇联1》中勾引鹰眼，《美队2》中勾引美国队长，在《妇联2》中终于……跟绿巨人表白了，黑寡妇用实际行动告诉了我们什么叫忠贞不二；而且为了治疗不孕不育连作战武器都变成了两支验孕棒(坚决相信快银没有死，后面还得回来)'
ch_text4 = ' 虽然从头打到尾，但是真的很无聊啊。'
ch_text5 = ' 剧情不如第一集好玩了，全靠密集笑点在提神。僧多粥少的直接后果就是每部寡姐都要换着队友谈恋爱，这特么比打斗还辛苦啊，真心求放过～～～（结尾彩蛋还以为是洛基呢，结果我呸！）'

ch_texts = [ch_text1, ch_text2, ch_text3, ch_text4, ch_text5]

In [54]:
import jieba

corpus = []
for ch_text in ch_texts:
    corpus.append(' '.join(jieba.cut(ch_text, cut_all=False)))

In [56]:
# 或者
corpus = [' '.join(jieba.cut(ch_text, cut_all=False)) for ch_text in ch_texts]

In [57]:
corpus

['  非常 失望 ， 剧本 完全 敷衍了事 ， 主线 剧情 没 突破 大家 可以 理解 ， 可 所有 的 人物 都 缺乏 动机 ， 正邪 之间 、 妇联 内部 都 没什么 火花 。 团结 - 分裂 - 团结 的 三段式 虽然 老套 但 其实 也 可以 利用 积攒 下来 的 形象 魅力 搞 出 意思 ， 但 剧本 写得 非常 肤浅 、 平面 。 场面 上 调度 混乱 呆板 ， 满屏 的 铁甲 审美疲劳 。 只有 笑 点算 得 上 差强人意 。',
 '  2015 年度 最 失望 作品 。 以为 面面俱到 ， 实则 画蛇添足 ； 以为 主题深刻 ， 实则 老调重弹 ； 以为 推陈出新 ， 实则 俗不可耐 ； 以为 场面 很 high ， 实则 high 劲 不足 。 气 ！ 上 一集 的 趣味 全无 ， 这集 的 笑 点 明显 刻意 到 心虚 。 全片 没有 任何 片段 给 我 有 紧张 激动 的 时候 ， 太弱 了 ， 跟 奥创 一样 。',
 '  《 铁人 2 》 中 勾引 钢铁 侠 ， 《 妇联 1 》 中 勾引 鹰眼 ， 《 美队 2 》 中 勾引 美国 队长 ， 在 《 妇联 2 》 中 终于 … … 跟 绿巨人 表白 了 ， 黑寡妇 用 实际行动 告诉 了 我们 什么 叫 忠贞不二 ； 而且 为了 治疗 不孕 不育 连 作战 武器 都 变成 了 两支 验孕 棒 ( 坚决 相信 快银 没有 死 ， 后面 还 得 回来 )',
 '  虽然 从头 打到 尾 ， 但是 真的 很 无聊 啊 。',
 '  剧情 不如 第一集 好玩 了 ， 全靠 密集 笑点 在 提神 。 僧多粥少 的 直接 后果 就是 每部 寡姐 都 要 换 着 队友 谈恋爱 ， 这特 么 比 打斗 还 辛苦 啊 ， 真心 求 放过 ～ ～ ～ （ 结尾 彩蛋 还 以为 是 洛基 呢 ， 结果 我 呸 ！ ）']

In [58]:
ch_vectorizer = TfidfVectorizer()
ch_feats = ch_vectorizer.fit_transform(corpus)

In [60]:
len(ch_vectorizer.get_feature_names())

136

In [61]:
ch_feats.toarray()[0, :]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.13332791,
        0.13332791,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.13332791,  0.        ,  0.13332791,
        0.13332791,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.13332791,  0.13332791,
        0.13332791,  0.13332791,  0.13332791,  0.        ,  0.10756811,
        0.26665581,  0.13332791,  0.        ,  0.        ,  0.13332791,
        0.26665581,  0.        ,  0.        ,  0.13332791,  0.        ,
        0.        ,  0.26665581,  0.10756811,  0.        ,  0.13332791,
        0.        ,  0.10756811,  0.        ,  0.        ,  0.10756811,
        0.13332791,  0.        ,  0.        ,  0.13332791,  0.        ,
        0.        ,  0.        ,  0.13332791,  0.13332791,  0.        ,
        0.13332791,  0.        ,  0.        ,  0.        ,  0.  